# Calculation of Higher and Lower Heating Values

This notebook shows how to compute the Higher Heating Value (HHV) and
Lower Heating Value (LHV) of a gaseous fuel using Cantera.

The main steps are:
- define the fuel composition in terms of Cantera species (here, pure H2),
- add the stoichiometric amount of oxygen (and nitrogen as inert air),
- compute the specific enthalpy of the reactants and products at standard
  conditions (298 K, 1 atm) and at chemical equilibrium,
- use a separate water phase to evaluate the latent heat of vaporization
  of water at 298 K,
- distinguish between HHV (water condensed to liquid) and LHV (water
  leaving as vapour).

Run the notebook, then modify the fuel composition to include CH4 or CO and
observe how the heating values per kg of fuel change.


In [7]:
import cantera as ct
import numpy as np

# --------------------------------------------------------------------------
# Define the conditions and state of the fuel
# --------------------------------------------------------------------------
gas = ct.Solution("gri30.yaml")

nsp = gas.n_species
names = gas.species_names

print(nsp)
print(names)
print()
print(gas.molecular_weights)
print("\n", np.zeros(nsp))

53
['H2', 'H', 'O', 'O2', 'OH', 'H2O', 'HO2', 'H2O2', 'C', 'CH', 'CH2', 'CH2(S)', 'CH3', 'CH4', 'CO', 'CO2', 'HCO', 'CH2O', 'CH2OH', 'CH3O', 'CH3OH', 'C2H', 'C2H2', 'C2H3', 'C2H4', 'C2H5', 'C2H6', 'HCCO', 'CH2CO', 'HCCOH', 'N', 'NH', 'NH2', 'NH3', 'NNH', 'NO', 'NO2', 'N2O', 'HNO', 'CN', 'HCN', 'H2CN', 'HCNN', 'HCNO', 'HOCN', 'HNCO', 'NCO', 'N2', 'AR', 'C3H7', 'C3H8', 'CH2CHO', 'CH3CHO']

[ 2.016  1.008 15.999 31.998 17.007 18.015 33.006 34.014 12.011 13.019
 14.027 14.027 15.035 16.043 28.01  44.009 29.018 30.026 31.034 31.034
 32.042 25.03  26.038 27.046 28.054 29.062 30.07  41.029 42.037 42.037
 14.007 15.015 16.023 17.031 29.022 30.006 46.005 44.013 31.014 26.018
 27.026 28.034 41.033 43.025 43.025 43.025 42.017 28.014 39.95  43.089
 44.097 43.045 44.053]

 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


In [2]:
iCH4 = gas.species_index("CH4")
io2 = gas.species_index("O2")
in2 = gas.species_index("N2")
ih2o = gas.species_index("H2O")
ico2 = gas.species_index("CO2")
ico = gas.species_index("CO")
iH2 = gas.species_index("H2")

MM = gas.molecular_weights  # Vector with molar masses of all species

# --------------------------------------------------------------------------
# Calculation of the LHV
# --------------------------------------------------------------------------
Tstp = 300  # STP temperature in K
Pstp = 101325  # STP pressure in Pa

# Initial composition vector
N = np.zeros(nsp)

# Define the fuel composition
N[iCH4] = 0
N[ico2] = 0
N[ico] = 0
N[iH2] = 1
N[ih2o] = 0
N[in2] = 0

gas.TPX = Tstp, Pstp, N
MM_fuel = gas.mean_molecular_weight
N_fuel = np.sum(N)

# Calculate stoichiometric oxygen requirement
#   H2 + 1/2 O2 --> H2O
#   CO + 1/2 O2 --> CO2
#   CH4 + 2*O2 --> CO2 + 2*H2O
N[io2] = N[iCH4] * 2 + N[iH2] * 0.5 + N[ico] * 0.5

# Mass of the mix (fuel + oxidizer)
mass_mix = np.dot(N, MM)

# Set the gas state
gas.TPX = Tstp, Pstp, N
MMmix = gas.mean_molecular_weight  # Mean molecular weight of the mix [kg_mix/kmol_mix]

# Enthalpy of the reagents [MJ/kg_mix]
HR = gas.enthalpy_mass / 1e6

# Equilibrate the mixture at constant TP
gas.equilibrate("TP")

# Enthalpy of the products [MJ/kg_mix]
HP = gas.enthalpy_mass / 1e6

# Adjust for latent heat of water
y_products = gas.Y  # Mass fractions of products
yH2O = y_products[ih2o]  # [kg_H2O/kg_mix]

water = ct.Water()
# Set liquid water state, with vapor fraction x = 0
water.TQ = 298, 0
h_liquid = water.h
# Set gaseous water state, with vapor fraction x = 1
water.TQ = 298, 1
h_gas = water.h
hfgH2O = (h_gas - h_liquid) / 1e6
print(hfgH2O)
# hfgH2O = 2.4379  # Latent heat for water at STP [MJ/kg]

# Enthalpy of the products accounting for water condensation [MJ/kg_mix]
HPc = HP - yH2O * hfgH2O

# Calculate the heating values
Qc = (HR - HPc) * mass_mix  # Heat of combustion [MJ]
HHV = Qc / (N_fuel * MM_fuel)  # Higher heating value [MJ/kg_fuel]

# Lower heating value
LHV = HHV - yH2O * hfgH2O * mass_mix / (N_fuel * MM_fuel)  # [MJ/kg_fuel]

# --------------------------------------------------------------------------
# Print Results
# --------------------------------------------------------------------------
print(f"Higher Heating Value (HHV): {HHV:.3f} MJ/kg_fuel")
print(f"Lower Heating Value (LHV): {LHV:.3f} MJ/kg_fuel")

2.4426665872392674
Higher Heating Value (HHV): 141.790 MJ/kg_fuel
Lower Heating Value (LHV): 119.962 MJ/kg_fuel
